The pdf We will summarize is [link text](https://arxiv.org/pdf/2008.12260.pdf)

In [127]:
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install pytorch_lightning

In [128]:
# import libraries
import torch
import re
import torch as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pytorch_lightning as pl
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
import os
import string
import torch.nn as nn
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import transformers
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers.optimization import AdamW
import torch.optim as optim
from transformers.optimization import get_linear_schedule_with_warmup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Read the data

In [129]:
data = pd.read_csv("/content/news_summary.csv")
data.head()

,headlines,text
0,upGrad learner switches to career in ML & AI w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [130]:
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
BATCH_size = 4
TEXT_LEN = 512
HEADLINE_LEN = 64
EPOCHS = 2
DEVICE = "cuda:0"

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [131]:
class NewsSummaryDataset(Dataset):
  def __init__(self, data, tokenizer, text_len, headkine_len):
    self.data = data
    self.headlines = self.data['headlines']
    self.text = self.data['text']
    self.tokenizer = tokenizer
    self.text_len = headline_len

  def __len__(self):
    return len(self.headlines)
  def __getitem__(self, idx):
    text = "summarize:" + str(self.ytext[idx])
    headline = str(self.headlines[idx])

    text_tokenizer = self.tokenizer(text, max_length=self.text_len, padding="max_length",
                                    truncation=True, add_special_tokens=True)
    headline_tokenizer = self.tokenizer(headline, max_length=self.headline_len, padding="max_length",
                                        truncation=True, add_special_tokens=True)

In [132]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(self,
                train_data,
                val_data,
                test_data,
                batch_size,
                tokenizer,
                text_len,
                summary_len):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.batch_size = batch_size
        self.test_data = test_data
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.summary_len = summary_len

    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(
            self.train_data,
            self.tokenizer,
            self.text_len,
            self.summary_len)

        self.val_dataset = NewsSummaryDataset(
            self.val_data,
            self.tokenizer,
            self.text_len,
            self.summary_len)

        self.test_dataset = NewsSummaryDataset(
            self.test_data,
            self.tokenizer,
            self.text_len,
            self.summary_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4)

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4)

    def test_dataloader(self):
        return DataLoader(
              self.test_dataset,
              batch_size=self.batch_size,
              num_workers=4
          )

In [133]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [134]:
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [135]:
news_summary_module = NewsSummaryDataModule(train_data, val_data, test_data, BATCH_SIZE, TOKENIZER, TEXT_LEN, HEADLINE_LEN)
news_summary_module.setup()

In [136]:

class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super(NewsSummaryModel, self).__init__()
        self.model = MODEL

    def forward(self, input_ids, attention_mask, labels=None, decoder_attention_mask=None):
        outputs = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        decoder_attention_mask=decoder_attention_mask)
        return outputs.loss, outputs.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        loss, output = self(input_ids=input_ids,
                      attention_mask=attention_mask)
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=0.0001)
        scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=0,
                num_training_steps=EPOCHS*len(data))
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [137]:
model = NewsSummaryModel()

In [146]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Specify the GPU ID you want to use

In [154]:
from pytorch_lightning import Trainer

trainer = Trainer(
    max_epochs=EPOCHS,
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [155]:
trainer.fit(model, NewsSummaryModel)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


TypeError: ignored

In [142]:
def summarize(text):
    inputs = TOKENIZER(text,
                       max_length=TEXT_LEN,
                       truncation=True,
                       padding="max_length",
                       add_special_tokens=True,
                       return_tensors="pt")
    summarized_ids = news_summary_model.model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_beams=4)

    return " ".join([TOKENIZER.decode(token_ids, skip_special_tokens=True)
                    for token_ids in summarized_ids])

In [143]:
bleu = evaluate.load("google_bleu")


In [151]:
text = data.iloc[100]["text"]
headline = data.iloc[100]["headlines"]
summarized_text = summarize(text)

NameError: ignored

In [145]:
bleu.compute(predictions=[summarized_text], references=[headline])

NameError: ignored